<a href="https://colab.research.google.com/github/jlb112/colab_deeplabcut/blob/main/video_labelling_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import pandas as pd
import numpy as np

In [ ]:
# Link to your GoogleDrive.

from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
project_folder_name = "2024Life/deeplabcut/buzz_attention_labelling"

In [ ]:
cap = cv2.VideoCapture('/content/drive/My Drive/2024Life/deeplabcut/buzz_attention_labelling/buzz_labeled.mp4')

In [ ]:
# Get video frame rate
fps = cap.get(cv2.CAP_PROP_FPS)
cap.release()
print(f"FPS = {fps}")

FPS = 30.004


In [ ]:
# Apply timestamp to csv
df = pd.read_csv("/content/drive/My Drive/2024Life/deeplabcut/buzz_attention_labelling/buzz_positions.csv")

df.head()

,frame_index,top_head_x,top_head_y,top_head_likelihood,mouth_x,mouth_y,mouth_likelihood,nose_tip_x,nose_tip_y,nose_tip_likelihood,right_eye_x,right_eye_y,right_eye_likelihood,left_eye_x,left_eye_y,left_eye_likelihood,LE_left_of_nose,RE_right_of_nose,attention
0,0,245.364070,163.63814,0.114846,465.34607,345.22890,0.165797,39.581398,492.03464,0.335487,272.112300,44.117104,0.258240,273.81552,47.310690,0.331277,N,Y,-1
1,1,259.030640,449.13965,0.127579,98.70082,710.91376,0.161946,32.242626,485.08746,0.528408,271.741970,44.089546,0.262830,273.54310,47.512100,0.386143,N,Y,-1
2,2,98.583880,486.54333,0.165682,83.21321,676.37463,0.157806,27.676983,473.68150,0.491913,121.180824,513.640440,0.358898,377.60675,62.028896,0.386726,N,Y,-1
3,3,115.329765,460.14850,0.159284,82.84120,646.07050,0.163060,455.792660,300.21518,0.201068,372.988130,60.230530,0.335501,378.16010,62.552197,0.389588,Y,N,-1
4,4,119.025910,134.17778,0.160454,67.41800,611.08360,0.157523,332.705930,112.59396,0.115846,375.229370,61.622300,0.326634,379.36460,63.717606,0.417935,N,Y,-1


In [ ]:
# create a new time column
df['time_s'] = df['frame_index'] / fps

# enabling filter or plot by time
print(df[['frame_index','time_s','attention']].head())

   frame_index    time_s  attention
0            0  0.000000         -1
1            1  0.033329         -1
2            2  0.066658         -1
3            3  0.099987         -1
4            4  0.133316         -1


In [ ]:
# Set attention as bool
df['attentive'] = df['attention'] == 1
df.set_index('frame_index', inplace=True)

In [ ]:
# 2) Open your source video and set up a writer
input_path  = '/content/drive/My Drive/2024Life/deeplabcut/buzz_attention_labelling/buzz_labeled.mp4'
output_path = '/content/drive/My Drive/2024Life/deeplabcut/buzz_attention_labelling/buzz_labeled_attention.mp4'

cap = cv2.VideoCapture(input_path)
fps = cap.get(cv2.CAP_PROP_FPS)
w   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (w,h))

In [ ]:
# 3) Iterate frames, overlay text/border, write out
frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_idx in df.index:
        row = df.loc[frame_idx]
        flag = row['attentive']
        color = (0,255,0) if flag else (0,0,255)  # green/red

        # a) Draw border
        cv2.rectangle(frame, (0,0), (w-1,h-1), color, thickness=5)

        # b) Put “Attention: Y/N”
        txt = f"Attention: {'Y' if flag else 'N'}"
        cv2.putText(frame, txt, (10, h-20),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2, cv2.LINE_AA)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print("Done! ▶️ Saved to", output_path)

Done! ▶️ Saved to /content/drive/My Drive/2024Life/deeplabcut/buzz_attention_labelling/buzz_labeled_attention.mp4
